In [1]:
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import mediapipe as mp
from fastdtw import fastdtw
from sklearn.preprocessing import normalize

# extract x, y from skeleton & change to (640 * 720)

In [2]:
def origin_rate(df):
    df_xy = df[[(a,b) for a,b in df.columns if b in ('x', 'y')]]
    df_xy.loc[:, (slice(None), 'x')] = df_xy.loc[:, (slice(None), 'x')].apply(lambda x : x*640)
    df_xy.loc[:, (slice(None), 'y')] = df_xy.loc[:, (slice(None), 'y')].apply(lambda x : x*720)
    return df_xy

# bounding box & perspective transform

In [3]:
def box_perspective(df_xy):
    for idx in range(len(df_xy)):
        x_min = df_xy.loc[idx, (slice(None), 'x')].min()
        x_max = df_xy.loc[idx, (slice(None), 'x')].max()
        y_min = df_xy.loc[idx, (slice(None), 'y')].min()
        y_max = df_xy.loc[idx, (slice(None), 'y')].max()
        
        if x_max != 0 and y_max != 0:
            x_rate = 640/((x_max+20)-(x_min-20))
            y_rate = 720/((y_max+20)-(y_min-20))

            df_xy.loc[idx, (slice(None), 'x')] = df_xy.loc[idx, (slice(None), 'x')].apply(lambda x : ((x-(x_min-20))*x_rate)/640)
            df_xy.loc[idx, (slice(None), 'y')] = df_xy.loc[idx, (slice(None), 'y')].apply(lambda x : ((x-(y_min-20))*y_rate)/720)
        
    return df_xy

# transform to eigen vector

In [4]:
def vector_transform(joint):    
    # Compute angles between joints
    v1 = joint[[13, 14, 15, 16, 17, 18, 23, 24, 25, 26, 27, 28, 29, 30], :] # Parent joint
    v2 = joint[[11, 12, 13, 14, 15, 16, 11, 12, 23, 24, 25, 26, 27, 28], :] # Child joint
    v = v2 - v1 # [14, 2]    
    # Normalize v
    v = v / np.linalg.norm(v, axis=1)[:, np.newaxis]
    
    return v

# cosine similarity

In [26]:
def cosine_similarity(train, test):
    cossim_list = []
    if not np.isnan(train).any() and np.isnan(test).any():
        return 0
    for x, y in zip(train, test):
        cossim = np.dot(x, y) / (np.linalg.norm(x)*np.linalg.norm(y))
        cossim_list.append(cossim)
    return np.nanmean(cossim_list)

# score by cosine similarity

In [27]:
def cossim_score(train, test):
    train = origin_rate(train)
    test = origin_rate(test)
    #print(train.shape, test.shape)
    
    train = box_perspective(train)
    test = box_perspective(test)
    #print(train.shape, test.shape)
    
    score_list=[]
    for idx in range(train.shape[0]):
        train_joint = train.iloc[idx].values.reshape(33,2)
        test_joint = test.iloc[idx].values.reshape(33,2)
        #print(train_joint)
        train_v = vector_transform(train_joint)
        test_v = vector_transform(test_joint)
        
        score = cosine_similarity(train_v, test_v)
        #print(score)
        score_list.append(score)
    print(score_list)
    
    return np.nanmean(score_list)

# euclidean distance of cosine similarity

In [28]:
def euclid_distance(train, test):
    euclid_list = []
    if not np.isnan(train).any() and np.isnan(test).any():
        return 0
    for x, y in zip(train, test):
        cossim = np.dot(x, y) / (np.linalg.norm(x)*np.linalg.norm(y))
        euc_d = np.sqrt(2*(1-cossim))
        score = 100 - (100 * (euc_d)/2)
        euclid_list.append(score)
    return np.nanmean(euclid_list)

# score by euclidean distance of cosine similarity

In [29]:
def euclid_score(train, test):
    train = origin_rate(train)
    test = origin_rate(test)
    
    train = box_perspective(train)
    test = box_perspective(test)
    
    score_list=[]
    for idx in range(train.shape[0]):
        train_joint = train.iloc[idx].values.reshape(33,2)
        test_joint = test.iloc[idx].values.reshape(33,2)

        train_v = vector_transform(train_joint)
        test_v = vector_transform(test_joint)

        score = euclid_distance(train_v, test_v)
        #print(score)
        score_list.append(score)
    print(score_list)
    
    return np.nanmean(score_list)

# fastdtw

In [30]:
def fast_dtw(train, test):
    if np.isnan(train).any():
        return np.nan
    
    if not np.isnan(train).any() and np.isnan(test).any():
        return 0
        
    return fastdtw(train,test)[0]

# score by fastdtw

In [31]:
def fastdtw_score(train, test):
    train = origin_rate(train)
    test = origin_rate(test)
    
    train = box_perspective(train)
    test = box_perspective(test)
    
    score_list=[]
    for idx in range(train.shape[0]):
        train_joint = train.iloc[idx].values.reshape(33,2)
        test_joint = test.iloc[idx].values.reshape(33,2)

        train_v = vector_transform(train_joint)
        test_v = vector_transform(test_joint)
        #print(train_v, train_v.shape, '*'*100, test_v, test_v.shape, sep='\n')
        score = fast_dtw(train_v, test_v)
        #print(score)
        score_list.append(score)
    print(score_list)
    return np.nanmean(score_list)

# load train, test data

In [11]:
verse3 = pd.read_csv("./image/verse3_skeleton.csv", header=[0,1])

# scoring with real dancing data

In [12]:
cam = pd.read_csv("./image/cam_skeleton.csv", header=[0,1])

### 1. cosine similarity

In [32]:
cossim_score(verse3, cam)

C:\Users\EOM\AppData\Local\Temp\ipykernel_25752\3483640641.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_xy.loc[:, (slice(None), 'x')] = df_xy.loc[:, (slice(None), 'x')].apply(lambda x : x*640)
C:\Users\EOM\AppData\Local\Temp\ipykernel_25752\3483640641.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_xy.loc[:, (slice(None), 'y')] = df_xy.loc[:, (slice(None), 'y')].apply(lambda x : x*720)
C:\Users\EOM\AppData\Local\Temp\ipykernel_25752\3483640641.py:3: SettingWithCopyWarning: 
A value is tryin

[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 0.8899328576945983, 0.8594182665314429, 0.8020233939474479, 0.7156818101807262, 0.626686507740926, 0.5425388134900356, 0.4997753957121209, 0.48032230400167336, 0.4303784537477054, 0.5512369632829603, 0.595353115113663, 0.6783874350681526, 0.9787501867273114, 0.9887016154630818, 0.9861806735058343, 0.9876200585283178, 0.9883287523754534, 0.9875940487166305, 0.9833773205402415, 0.9841969411396727, 0.9846882199581085, 0.9827441802592459, 0.8002947891503908, 0.7338011359918084, 0.7018960135158208, 0.6972685014426337, 0.7038313568463529, 0.7104751276275884, 0.697117024102577, 0.6958413259970995, 0.6919317807106411, 0.6823124630125277, 0.6871912045057955, 0.6859792386442434, 0.6972693385501783, 0.6979448607045727, 0.704978116282714, 0.721175585426512

C:\Users\EOM\AppData\Local\Temp\ipykernel_25752\2836190961.py:7: RuntimeWarning: invalid value encountered in true_divide
  v = v / np.linalg.norm(v, axis=1)[:, np.newaxis]
C:\Users\EOM\AppData\Local\Temp\ipykernel_25752\3700788703.py:8: RuntimeWarning: Mean of empty slice
  return np.nanmean(cossim_list)


0.798394500948187

- 0.798394500948187

### 2. fastdtw

In [24]:
fastdtw_score(verse3, cam)

C:\Users\EOM\AppData\Local\Temp\ipykernel_25752\3483640641.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_xy.loc[:, (slice(None), 'x')] = df_xy.loc[:, (slice(None), 'x')].apply(lambda x : x*640)
C:\Users\EOM\AppData\Local\Temp\ipykernel_25752\3483640641.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_xy.loc[:, (slice(None), 'y')] = df_xy.loc[:, (slice(None), 'y')].apply(lambda x : x*720)
C:\Users\EOM\AppData\Local\Temp\ipykernel_25752\3483640641.py:3: SettingWithCopyWarning: 
A value is tryin

[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 7.285869558453565, 8.147188097353975, 9.565665125470758, 11.220285039860459, 12.50813765560413, 13.374825666051406, 13.751164184303942, 12.651947102891334, 12.385793187602067, 11.15230771472367, 8.65084393603996, 7.988105084667746, 2.855626626231299, 2.167888753222865, 2.3189262742930383, 2.14146595129032, 2.1057991369400284, 2.154739667778656, 2.5350356063369537, 2.543416242941509, 2.39416869208101, 2.6759230934254084, 6.309757077920649, 7.4952912097459095, 7.740017054581349, 8.056677492498125, 7.684344322908323, 7.751140444812601, 7.776873719278649, 7.512874836411807, 7.585976376485268, 7.767152454799167, 7.808362541872468, 7.797602840732477, 7.165900931404065, 6.928007010538171, 6.559928088172189, 7.497283055407238, 6.949064643431213, 7.9272

6.845611668202794

- 6.845611668202794

### 3. euclidean distance of cosine similarity

In [25]:
euclid_score(verse3, cam)

C:\Users\EOM\AppData\Local\Temp\ipykernel_25752\3483640641.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_xy.loc[:, (slice(None), 'x')] = df_xy.loc[:, (slice(None), 'x')].apply(lambda x : x*640)
C:\Users\EOM\AppData\Local\Temp\ipykernel_25752\3483640641.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_xy.loc[:, (slice(None), 'y')] = df_xy.loc[:, (slice(None), 'y')].apply(lambda x : x*720)
C:\Users\EOM\AppData\Local\Temp\ipykernel_25752\3483640641.py:3: SettingWithCopyWarning: 
A value is tryin

[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 80.427504973955, 78.24961164270734, 74.65992381889399, 70.24530773276543, 66.5632628293912, 63.452393910548736, 62.01683305258666, 62.68189736051447, 62.3091892930496, 67.69359898474276, 70.65024492625103, 75.71016614336433, 91.83476148762384, 93.662114001843, 93.18551672789218, 93.62613847260293, 93.8164989967908, 93.69470079481528, 92.65747822060645, 92.59776776145827, 92.96107617475958, 92.22526253302354, 79.99526554652815, 77.37607017646762, 76.58653193414435, 76.10192627375261, 76.75148370543032, 76.93837057848454, 77.60720787298148, 78.08933752532559, 77.5288813542459, 76.79076513834939, 76.95635670292855, 76.77225976013544, 77.02877062114837, 77.06292372673632, 76.17241899826566, 75.90315639952654, 80.11794442312338, 77.16703161711314, 6

C:\Users\EOM\AppData\Local\Temp\ipykernel_25752\2836190961.py:7: RuntimeWarning: invalid value encountered in true_divide
  v = v / np.linalg.norm(v, axis=1)[:, np.newaxis]
C:\Users\EOM\AppData\Local\Temp\ipykernel_25752\1633764670.py:10: RuntimeWarning: Mean of empty slice
  return np.nanmean(euclid_list)


79.40835738580742

- 79.40835738580742

# scoring with different data

In [16]:
mjbj = pd.read_csv("./image/mjbj2_skeleton.csv", header=[0,1])

### 1. cosine similarity

In [33]:
cossim_score(mjbj, cam)

C:\Users\EOM\AppData\Local\Temp\ipykernel_25752\3483640641.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_xy.loc[:, (slice(None), 'x')] = df_xy.loc[:, (slice(None), 'x')].apply(lambda x : x*640)
C:\Users\EOM\AppData\Local\Temp\ipykernel_25752\3483640641.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_xy.loc[:, (slice(None), 'y')] = df_xy.loc[:, (slice(None), 'y')].apply(lambda x : x*720)
C:\Users\EOM\AppData\Local\Temp\ipykernel_25752\3483640641.py:3: SettingWithCopyWarning: 
A value is tryin

[nan, 0.76759886520883, 0.7294872980211877, 0.7153093313824567, 0.7147501441954242, 0.7173225718260742, 0.7302887451723002, 0.7351345084051, 0.7742153138105946, 0.786265204622819, 0.800915373083482, 0.8076417073318994, 0.822991063662616, 0.8501150865236414, 0.848880165728413, 0.8613887417999463, 0.8602901890548379, 0.8531536222546284, 0.8598436805405765, 0.8722993956333955, 0.8730901984002689, 0.874364024061305, 0.8785853745915393, 0.8775575063203586, 0.8791224700140633, 0.8920933513211011, 0.8493047368867437, 0.8240403892819069, 0.81599137526913, 0.8027913716736732, 0.801587157540175, 0.8008905989831344, 0.7993856878921213, 0.8033219605455951, 0.8013204475685366, 0.8013247181125108, 0.7990418746946971, 0.7972255534852379, 0.7996143058975681, 0.8058544657392729, 0.8122310222990894, 0.8137349269976566, 0.8195112532918445, 0.8178232092656825, 0.8268602304102097, 0.8238080404429458, 0.8267115257734152, 0.8312103233857682, 0.8322074177459406, 0.8347668388743469, 0.8377249157226622, 0.84241

C:\Users\EOM\AppData\Local\Temp\ipykernel_25752\2836190961.py:7: RuntimeWarning: invalid value encountered in true_divide
  v = v / np.linalg.norm(v, axis=1)[:, np.newaxis]
C:\Users\EOM\AppData\Local\Temp\ipykernel_25752\3700788703.py:8: RuntimeWarning: Mean of empty slice
  return np.nanmean(cossim_list)


0.5607927518205723

- 0.5607927518205723

### 2. fastdtw

In [34]:
fastdtw_score(mjbj, cam)

C:\Users\EOM\AppData\Local\Temp\ipykernel_25752\3483640641.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_xy.loc[:, (slice(None), 'x')] = df_xy.loc[:, (slice(None), 'x')].apply(lambda x : x*640)
C:\Users\EOM\AppData\Local\Temp\ipykernel_25752\3483640641.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_xy.loc[:, (slice(None), 'y')] = df_xy.loc[:, (slice(None), 'y')].apply(lambda x : x*720)
C:\Users\EOM\AppData\Local\Temp\ipykernel_25752\3483640641.py:3: SettingWithCopyWarning: 
A value is tryin

[nan, 6.032490836174933, 8.416494438104435, 7.93496563719164, 6.481493625427639, 7.950996554854312, 7.842752751295846, 7.942977666390469, 7.449151833184072, 7.186552806536575, 7.3246837457096206, 7.277540082570315, 7.389684930849817, 6.632500560530385, 6.950489886681665, 7.286480747264536, 6.706285437166182, 7.0576543371125435, 6.814649936002395, 6.477915331472641, 6.603964686353133, 6.816818866610912, 6.491154617388651, 6.456511756094759, 6.56217317530084, 6.358388680322492, 6.244677382190538, 6.156777106295165, 6.409876552103015, 6.66620706369231, 6.821372724918634, 7.07360019232404, 7.190301382253785, 6.941950010413526, 7.029355223216105, 6.9585064099023635, 7.023344743132139, 7.130725296688514, 6.906493861501127, 6.699743576702459, 6.422330983998248, 6.275372112403956, 5.895915517091254, 5.951661099042047, 5.867058859524711, 5.813995879360458, 5.778643775978386, 5.821805291591427, 5.832010285731352, 5.726325218433159, 5.691647557599988, 5.911596166158914, 6.067368511261337, 6.36395

10.675869959856124

- 10.675869959856124

### 3. euclidean distance of cosine similarity

In [35]:
euclid_score(mjbj, cam)

C:\Users\EOM\AppData\Local\Temp\ipykernel_25752\3483640641.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_xy.loc[:, (slice(None), 'x')] = df_xy.loc[:, (slice(None), 'x')].apply(lambda x : x*640)
C:\Users\EOM\AppData\Local\Temp\ipykernel_25752\3483640641.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_xy.loc[:, (slice(None), 'y')] = df_xy.loc[:, (slice(None), 'y')].apply(lambda x : x*720)
C:\Users\EOM\AppData\Local\Temp\ipykernel_25752\3483640641.py:3: SettingWithCopyWarning: 
A value is tryin

[nan, 72.6649513905453, 70.64872881858342, 69.75817745050664, 68.89808310470235, 69.19169811441536, 71.04704544860354, 71.26698955305622, 73.30133601784847, 74.06562152129568, 75.04078615919526, 75.65088382925948, 76.68767483808537, 79.54960638766367, 78.31172150144558, 78.16531183438349, 79.58710184144104, 78.72528345947883, 79.43677899898991, 80.07349800033505, 79.7328653433518, 79.50097698299898, 79.86685077533025, 79.9098222492123, 79.85485231300656, 80.2210977440199, 78.57685090002254, 76.533243520787, 75.62613679278309, 74.6292784379569, 74.34859556647233, 74.06763073839082, 74.03732310417487, 74.58961677729162, 74.3621816050307, 74.33027697663104, 74.1450638818237, 73.95056011464365, 74.23867998258787, 75.0089164656154, 75.69357437330099, 75.69003522183822, 76.53588130718539, 76.80128060452333, 78.23571553830725, 78.07919510720741, 78.45463308245091, 78.6834645410235, 78.60078977066439, 78.74174095647616, 78.94867701764697, 78.99091367191336, 78.75136930042977, 78.34439399249683

C:\Users\EOM\AppData\Local\Temp\ipykernel_25752\2836190961.py:7: RuntimeWarning: invalid value encountered in true_divide
  v = v / np.linalg.norm(v, axis=1)[:, np.newaxis]
C:\Users\EOM\AppData\Local\Temp\ipykernel_25752\2344704103.py:10: RuntimeWarning: Mean of empty slice
  return np.nanmean(euclid_list)


63.61107212259936

- 63.61107212259936

# scoring with null data

In [38]:
df_nan = pd.DataFrame(np.full((382,132), np.nan), columns=cam.columns)

In [39]:
df_nan.head()

NOSE             LEFT_EYE_INNER             LEFT_EYE      ... RIGHT_HEEL  \
     x   y   z   v              x   y   z   v        x   y  ...          z   
0  NaN NaN NaN NaN            NaN NaN NaN NaN      NaN NaN  ...        NaN   
1  NaN NaN NaN NaN            NaN NaN NaN NaN      NaN NaN  ...        NaN   
2  NaN NaN NaN NaN            NaN NaN NaN NaN      NaN NaN  ...        NaN   
3  NaN NaN NaN NaN            NaN NaN NaN NaN      NaN NaN  ...        NaN   
4  NaN NaN NaN NaN            NaN NaN NaN NaN      NaN NaN  ...        NaN   

      LEFT_FOOT_INDEX             RIGHT_FOOT_INDEX              
    v               x   y   z   v                x   y   z   v  
0 NaN             NaN NaN NaN NaN              NaN NaN NaN NaN  
1 NaN             NaN NaN NaN NaN              NaN NaN NaN NaN  
2 NaN             NaN NaN NaN NaN              NaN NaN NaN NaN  
3 NaN             NaN NaN NaN NaN              NaN NaN NaN NaN  
4 NaN             NaN NaN NaN NaN              NaN NaN NaN NaN  

[5 rows x 132 columns]

### 1. cosine similarity

In [40]:
cossim_score(verse3, df_nan)

C:\Users\EOM\AppData\Local\Temp\ipykernel_25752\3483640641.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_xy.loc[:, (slice(None), 'x')] = df_xy.loc[:, (slice(None), 'x')].apply(lambda x : x*640)
C:\Users\EOM\AppData\Local\Temp\ipykernel_25752\3483640641.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_xy.loc[:, (slice(None), 'y')] = df_xy.loc[:, (slice(None), 'y')].apply(lambda x : x*720)
C:\Users\EOM\AppData\Local\Temp\ipykernel_25752\3483640641.py:3: SettingWithCopyWarning: 
A value is tryin

[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

C:\Users\EOM\AppData\Local\Temp\ipykernel_25752\2836190961.py:7: RuntimeWarning: invalid value encountered in true_divide
  v = v / np.linalg.norm(v, axis=1)[:, np.newaxis]
C:\Users\EOM\AppData\Local\Temp\ipykernel_25752\3700788703.py:8: RuntimeWarning: Mean of empty slice
  return np.nanmean(cossim_list)


0.0

- 0.0

### 2. fastdtw

In [41]:
fastdtw_score(verse3, df_nan)

C:\Users\EOM\AppData\Local\Temp\ipykernel_25752\3483640641.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_xy.loc[:, (slice(None), 'x')] = df_xy.loc[:, (slice(None), 'x')].apply(lambda x : x*640)
C:\Users\EOM\AppData\Local\Temp\ipykernel_25752\3483640641.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_xy.loc[:, (slice(None), 'y')] = df_xy.loc[:, (slice(None), 'y')].apply(lambda x : x*720)
C:\Users\EOM\AppData\Local\Temp\ipykernel_25752\3483640641.py:3: SettingWithCopyWarning: 
A value is tryin

[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

C:\Users\EOM\AppData\Local\Temp\ipykernel_25752\2836190961.py:7: RuntimeWarning: invalid value encountered in true_divide
  v = v / np.linalg.norm(v, axis=1)[:, np.newaxis]


0.0

- 0.0

### 3. euclidean distance of cosine similarity

In [42]:
euclid_score(verse3, df_nan)

C:\Users\EOM\AppData\Local\Temp\ipykernel_25752\3483640641.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_xy.loc[:, (slice(None), 'x')] = df_xy.loc[:, (slice(None), 'x')].apply(lambda x : x*640)
C:\Users\EOM\AppData\Local\Temp\ipykernel_25752\3483640641.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_xy.loc[:, (slice(None), 'y')] = df_xy.loc[:, (slice(None), 'y')].apply(lambda x : x*720)
C:\Users\EOM\AppData\Local\Temp\ipykernel_25752\3483640641.py:3: SettingWithCopyWarning: 
A value is tryin

[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

C:\Users\EOM\AppData\Local\Temp\ipykernel_25752\2836190961.py:7: RuntimeWarning: invalid value encountered in true_divide
  v = v / np.linalg.norm(v, axis=1)[:, np.newaxis]
C:\Users\EOM\AppData\Local\Temp\ipykernel_25752\2344704103.py:10: RuntimeWarning: Mean of empty slice
  return np.nanmean(euclid_list)


0.0

- 0.0

# 정리

### 실제 동일하게 춤을 춘 경우

- cosine similarity : 0.798394500948187
- fastdtw : 6.845611668202794
- euclidean distance of cosine similarity : 79.40835738580742

### 다른 동작일 경우

- cosine similarity : 0.5607927518205723
- fastdtw : 10.675869959856124
- euclidean distance of cosine similarity : 63.61107212259936

### null data인 경우

- cosine similarity, fastdtw, euclidean distance of similarity : 모두 0.0

# 결론

- 벡터 값을 추출하여 preprocessing을 했을 때 그 전에 비해 성능이 많이 올라간 것을 알 수 있음
- 다른 동작을 했을 경우에 틀렸다고 표현해내는 부분이 많이 개선되었음
- 벡터 값을 이용해서 각도를 뽑아내는 과정에서 각도 선택을 추가적으로 하는 방향을 고려해봐야함
- 여러가지를 테스트 해보고 점수의 최소값과 최대값을 정하는 작업이 필요함